In [1]:
from cyschoolhousesuite import *
import xlwings as xw
from PyPDF2 import PdfFileMerger

logf = open(log_path + '//SY18_Smart_Trackers_Update.log', "w")
logf.write("{0} --- starting script {1}\n".format(pd.to_datetime('now'), "SY18 Smart Trackers Update"))

df = get_report(report_key = '00O1a000002rdvJ')

df.sort_values(by=["Account Name", "Intervention Primary Staff: Staff Name", 
                   "Program: Program Name", "Student: Student Name"], inplace = True)

df.loc[df["Program: Program Name"] == 'Tutoring: Literacy', "Program: Program Name"] = "ELA"
df.loc[df["Program: Program Name"] == 'Tutoring: Math', "Program: Program Name"] = "Math"
df['Student/Program: Total Time for Program'].fillna(value=0, inplace=True)
df['Student/Program: Total Time for Program'] = df['Student/Program: Total Time for Program'].astype(int)

# Load School Data
sch_ref = "Z:\\ChiPrivate\\Chicago Data and Evaluation\\SY18\\SY18 Team Placement\\SY18 FINAL Team Placements.xlsx"
sch_ref_df = pd.read_excel(sch_ref, sheetname="School Data")
sch_ref_df = sch_ref_df[~(sch_ref_df['School'].isnull())]

# Open Excel Template
xlsx_path = Path(templates_path) / 'CHI - Smart Tracker Template.xlsx'

wb = xw.Book(str(xlsx_path))
sht = wb.sheets['SMART Tracker']
sht0 = wb.sheets['Header']
sht1 = wb.sheets['Course Performance']
sht2 = wb.sheets['SEL']
sht3 = wb.sheets['Attendance Tracker']
sht4 = wb.sheets['Attendance CICO']

# Iterate through school names to build Smart Tracker .pdfs
pdfs_path = Path.cwd().parents[1] / 'cyautomation/cyschoolhouse/temp'

for x in sch_ref_df['Informal School Name']:
    i =+ 1
    logf.write("{0} of {1} - {2}\n".format(i, len(sch_ref_df), x))
    
    if not (pdfs_path / x).exists():
        (pdfs_path / x).mkdir()
    
    sch_cyc_fldr = sch_ref_df['cyconnect Folder'][sch_ref_df['Informal School Name']==x].values[0]    
    school = sch_ref_df['School'][sch_ref_df['Informal School Name'] == x].values[0]
    df_school = df.loc[df["Account Name"] == school]
    
    for y in df_school["Intervention Primary Staff: Staff Name"].unique():
        sht0.range('B1').options(index=False, header=False).value = y

        df_acm = df_school[df_school["Intervention Primary Staff: Staff Name"] == y]
        # Course Performance
        df_acm_tutoring = df_acm[df_acm["Program: Program Name"].str.contains("Math") | df_acm["Program: Program Name"].str.contains("ELA")]
        df_acm_tutoring['Student/Program: Total Time for Program'] = df_acm_tutoring['Student/Program: Total Time for Program'].astype(str) + "\r\n" + df_acm_tutoring["Program: Program Name"]
        df_acm_tutoring = df_acm_tutoring[["Student: Student Name", 'Student/Program: Total Time for Program']]
        if len(df_acm_tutoring) > 12:
            logf.write("Warning: More than 12 Math/ELA students for {0}\n".format(y))
        sht1.range('B4:C15').clear_contents()
        sht1.range('B4').options(index=False, header=False).value = df_acm_tutoring[0:12]
        # SEL
        df_acm_SEL = df_acm[df_acm["Program: Program Name"].str.contains("SEL")]
        if len(df_acm_SEL) > 6:
            logf.write("Warning: More than 6 SEL students for {0}\n".format(y))
        sht2.range('B5:B10').clear_contents()
        sht2.range('B5').options(index=False, header=False).value = df_acm_SEL["Student: Student Name"][0:6]
        # Attendance
        df_acm_attendance = df_acm[df_acm["Program: Program Name"].str.contains("Attendance")]
        sht4.range('B4:B6').clear_contents()
        sht4.range('F4:F6').clear_contents()
        if len(df_acm_attendance["Student: Student Name"]) > 6:
            logf.write("Warning: More than 6 Attendance students for {0}\n".format(y))
        sht4.range('B4').options(index=False, header=False).value = df_acm_attendance["Student: Student Name"][0:3]
        sht4.range('F4').options(index=False, header=False).value = df_acm_attendance["Student: Student Name"][3:6]
        
        # Export PDF
        try:
            sht.api.ExportAsFixedFormat(0, str(pdfs_path / x / (y + ".pdf")))
        except:
            logf.write("Error converting to pdf for {0}\n".format(y))
            pass
    
    # Merge team PDFs and write PDF to cyconnect
    merger = PdfFileMerger()
    for file in (pdfs_path / x).iterdir():
        merger.append(str(file))
    merger.write("Z:\\" + sch_cyc_fldr + "\\" + "SY18 Smart Trackers - " + x + ".pdf")
    merger.close()
    
    # Remove local data (function comes from cyschoolhousesuite.py)
    delete_folder(pdfs_path / x)
    
wb.close()
logf.write("Completed script 'SY18 SMART Trackers Update'\n")
logf.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
